In [434]:
# Commment for Mumu 
# This block will read User Item interaction Json
# Attention, till now the Id is still in the raw format, which means it is still the steam format id
# Our goal is to map the User/User/Bundle into its range of size.
# Like say a table is like 
# | UserId | ItemId |
# | 1121241414124 | 10 |
# | 1121241414124 | 20 |
# | 1121241414123 | 10 |
# | 1121241414122 | 30 |
# | 1121241414122 | 20 | 

# it will become 
# | UserId | ItemId |
# | 1 | 1 |
# | 1 | 2 |
# | 2 | 1 |
# | 3 | 3 |
# | 3 | 2 | 


import pandas as pd
import json
new_clean_user_item = pd.read_json(open("../data/clean_user_item.json"))
new_clean_user_item.rename({0: "user", 1: "item_id", 2: "total_game_times", 3: "game_time_within_2weeks"}, inplace=True,
                           axis=1)
new_clean_user_item["user"] = new_clean_user_item["user"].astype(str)
print("finish renaming")
new_clean_user_item=new_clean_user_item.drop(["total_game_times", "game_time_within_2weeks"], axis=1)
new_clean_user_item["item_id"] = new_clean_user_item["item_id"].astype(str)
new_clean_user_item['user'] = new_clean_user_item['user'].astype(str)
print("finish storing")
new_clean_user_item.head()

finish renaming
finish storing


,user,item_id
0,76561197970982480,10
1,76561197970982480,30
2,76561197970982480,300
3,76561197970982480,240
4,76561197970982480,3830


In [4]:
import pandas as pd
new_clean_user_item = pd.read_json(open("../data/clean_user_item.json"))
new_clean_user_item.rename({0: "user", 1: "item_id", 2: "total_game_times", 3: "game_time_within_2weeks"}, inplace=True,
                           axis=1)

In [11]:
new_clean_user_item

,user,item_id,total_game_times,game_time_within_2weeks
0,76561197970982480,10,6,0
1,76561197970982480,30,7,0
2,76561197970982480,300,4733,0
3,76561197970982480,240,1853,0
4,76561197970982480,3830,333,0
...,...,...,...,...
3285241,76561198329548336,304930,677,677
3285242,76561198329548336,227940,43,43
3285243,76561198329548336,388490,3,3
3285244,76561198329548336,521570,4,4


In [435]:
# Get the Bundle : list<Item> data 
raw_bundle_item = pd.read_json("../data/bundle_data.json")


In [436]:
# In this block, we extract item data from the previous DF. 
raw_bundle_item = raw_bundle_item
item_data = []
for bundle_id in raw_bundle_item.bundle_id.unique():
    step_bundle_item = raw_bundle_item[raw_bundle_item["bundle_id"] == bundle_id]
    item_list = step_bundle_item["items"].item()
    for item in item_list:
        item['bundle_id'] = bundle_id
    item_data += item_list
complex_item_data = item_data_df[item_data_df["item_id"].str.contains(",")].copy()
item_data_df = item_data_df.drop(item_data_df[item_data_df["item_id"].str.contains(",")].index)
complex_item_data
new_item_data = []
for index,row in complex_item_data.iterrows():
    item_id_list = row["item_id"].split(",")
    for item_id in item_id_list:
        new_row = row.copy()
        new_row["item_id"] = item_id
        new_item_data.append(new_row)
new_item_data_df = pd.DataFrame(new_item_data)
item_data_df = pd.concat([item_data_df, new_item_data_df])
item_data_df = item_data_df.drop_duplicates()

In [437]:
# In this block we get the bundle item mapping
# Attention, in the previous user item mapping,
# we have map the item_id from its actual ID(10,20,30) into their sequence id, 
# we will use the seq id of item for bundle_item as well
# but at this point we haven't done it. 
# We now have the actual bundle id and item id 
bundle_item = item_data_df[["bundle_id", "item_id"]]
bundle_item['bundle_id'] = bundle_item['bundle_id'].astype(str)
bundle_item['item_id'] = bundle_item['item_id'].astype(str)
item_data_df["item_id"] = item_data_df["item_id"].astype(str)
bundle_item

/var/folders/t4/40312mt171v54r6r4wj8l2xm0000gn/T/ipykernel_4966/3753812211.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bundle_item['bundle_id'] = bundle_item['bundle_id'].astype(str)
/var/folders/t4/40312mt171v54r6r4wj8l2xm0000gn/T/ipykernel_4966/3753812211.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bundle_item['item_id'] = bundle_item['item_id'].astype(str)


,bundle_id,item_id
0,450,326950
1,450,331490
2,450,331491
3,450,331492
4,450,348540
...,...,...
2944,308,233700
2944,308,238450
2944,308,258930
3316,445,365660


In [438]:

# To create user bundle map 
# Attention! I cannot find the direct user bundle interaction data, so I used the merge. 
user_bundle = pd.merge(new_clean_user_item, bundle_item, how='left', on = ["item_id"])
user_bundle = user_bundle.drop_duplicates(["user","item_id"])
# Question! 
# We can keep only 1 user_bundle interaction record, if it's merged on the same item id
# E.G.
# user | item_id | bundle_id
# 76561197970982480 | 30 | 232
# 76561197970982480 | 30 | 235 # considered as duplicated
# 76561197970982480 | 300 | 232
# 76561197970982480 | 300 | 240 # considered as duplicated

# we can drop the 2 lines 
# The remain will be like 
# user | item_id | bundle_id
# 76561197970982480 | 30 | 232
# 76561197970982480 | 300 | 232


In [439]:
user_bundle = user_bundle.dropna()
user_bundle = user_bundle[["user", "bundle_id"]]
user_bundle = user_bundle.drop_duplicates()
user_bundle

,user,bundle_id
0,76561197970982480,232
10,76561197970982480,957
12,76561197970982480,574
15,76561197970982480,362
16,76561197970982480,572
...,...,...
3693069,76561198312638240,1158
3693074,76561198313357712,232
3693076,76561198313471776,232
3693078,76561198314015552,231


In [440]:
user_bundle['user'] = user_bundle['user'].astype(str)
user_bundle["bundle_id"] = user_bundle["bundle_id"].astype(str)
user_bundle["bundle_id"] = user_bundle["bundle_id"].apply(lambda x: x.replace(".0",""))

In [441]:
# In this section we will have the actual Id map to seq_id map! 

from collections import defaultdict
user_mapping = defaultdict(str)
bundle_mapping = defaultdict(str)
item_mapping = defaultdict(str)

user_id_list = sorted(list(set(list(new_clean_user_item.user.unique()) + list(user_bundle.user.unique()))))
bundle_id_list = sorted(list(set(list(user_bundle.bundle_id.unique()) + list(bundle_item.bundle_id.unique()))))
item_id_list = sorted(list(set(list(new_clean_user_item.item_id.unique()) + list(bundle_item.item_id.unique()))))

for index, uid in enumerate(user_id_list):
    user_mapping[uid] = str(index)
for index, bid in enumerate(bundle_id_list):
    bundle_mapping[bid] = str(index)
for index, iid in enumerate(item_id_list):
    item_mapping[iid] = str(index)

In [442]:
# Saving the mapping, and then we can use it for calculating the score. 
user_mapping_json = json.dumps(user_mapping)
with open("user_mapping_json.json",'w') as f:
    f.write(user_mapping_json)
    
bundle_mapping_json = json.dumps(bundle_mapping)
with open("bundle_mapping_json.json",'w') as f:
    f.write(bundle_mapping_json)

item_mapping_json = json.dumps(item_mapping)
with open("item_mapping_json.json",'w') as f:
    f.write(item_mapping_json)

In [443]:
# TO MAP the ID 
user_bundle['user'] = user_bundle['user'].apply(lambda x: user_mapping[x])
user_bundle['bundle_id'] = user_bundle['bundle_id'].apply(lambda x: bundle_mapping[x])
bundle_item['bundle_id'] = bundle_item['bundle_id'].apply(lambda x: bundle_mapping[x])
bundle_item['item_id'] = bundle_item['item_id'].apply(lambda x: item_mapping[x])
new_clean_user_item['user'] = new_clean_user_item['user'].apply(lambda x: user_mapping[x])
new_clean_user_item['item_id'] = new_clean_user_item['item_id'].apply(lambda x: item_mapping[x])

/var/folders/t4/40312mt171v54r6r4wj8l2xm0000gn/T/ipykernel_4966/1000420821.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bundle_item['bundle_id'] = bundle_item['bundle_id'].apply(lambda x: bundle_mapping[x])
/var/folders/t4/40312mt171v54r6r4wj8l2xm0000gn/T/ipykernel_4966/1000420821.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bundle_item['item_id'] = bundle_item['item_id'].apply(lambda x: item_mapping[x])


In [413]:
user_bundle.to_csv("user_bundle.txt", sep='\t', header = False, index = False)
bundle_item.to_csv("bundle_item.txt",sep='\t', header=False, index=False)
new_clean_user_item.to_csv("user_item.txt",sep='\t', header=False, index=False)

In [414]:
# Different ways to divide the train_set, keep every user, but select a part of their interactions for training.


In [415]:
train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10
# user_bundle_train = user_bundle.sample(frac=0.75) # This part can be used in different ways 
user_bundle_train = user_bundle.groupby('user').apply(lambda x: x.sample(frac=0.75, random_state=1)) # using this will select user at least once
user_bundle_train.to_csv("user_bundle_train.txt", sep='\t', header = False, index = False)

In [416]:
user_bundle_remain = pd.concat([user_bundle_train, user_bundle]).drop_duplicates(keep=False)
user_bundle_test = user_bundle_remain.sample(frac=0.6)
user_bundle_tuning = pd.concat([user_bundle_remain, user_bundle_test]).drop_duplicates(keep=False)
user_bundle_test.to_csv("user_bundle_test.txt", sep='\t', header=False, index=False)
user_bundle_tuning.to_csv("user_bundle_tune.txt", sep='\t', header=False, index=False)

In [426]:
len(new_clean_user_item.item_id.unique())

10050

In [428]:
new_clean_user_item.item_id.astype(int).max()

11515